# Advanced Operations Example

This example uses the user_assessments hdfs file from RandomDataset. User assessments file contains a user table and a assessments table, that imitate the data structure of in CSS (Covid Symptom Study) project.

In [1]:
!ls *hdf5

dataset.hdf5  temp2.hdf5  temp.hdf5  user_assessments.hdf5


In [6]:
from exetera.core.session import Session
s = Session()  # not recommended, but to cover all the cells in the example, we use this way here
src = s.open_dataset('user_assessments.hdf5', 'r', 'src')
print(src.keys())
users = src['users']
print('Columns in users table:', users.keys())
# use describe to check the value in each column
users.describe(include=['bmi', 'has_diabetes', 'height_cm',  'year_of_birth'])

dict_keys(['assessments', 'users'])
Columns in users table: odict_keys(['FirstName', 'LastName', 'bmi', 'bmi_valid', 'has_diabetes', 'height_cm', 'height_cm_valid', 'id', 'j_valid_from', 'j_valid_to', 'year_of_birth', 'year_of_birth_valid'])
fields	            bmi	   has_diabetes	      height_cm	  year_of_birth	
count	             10	             10	             10	             10	
unique	            NaN	              1	            NaN	            NaN	
top	            NaN	              0	            NaN	            NaN	
freq	            NaN	             10	            NaN	            NaN	
mean	          31.70	            NaN	         135.60	        1965.40	
std	           5.14	            NaN	          25.39	          24.87	
min	          25.00	            NaN	         107.00	        1926.00	
25%	          25.02	            NaN	         107.20	        1926.07	
50%	          25.05	            NaN	         107.41	        1926.13	
75%	          25.07	            NaN	         107.61	      

{'fields': ['bmi', 'has_diabetes', 'height_cm', 'year_of_birth'],
 'count': [10, 10, 10, 10],
 'mean': ['31.70', 'NaN', '135.60', '1965.40'],
 'std': ['5.14', 'NaN', '25.39', '24.87'],
 'min': ['25.00', 'NaN', '107.00', '1926.00'],
 '25%': ['25.02', 'NaN', '107.20', '1926.07'],
 '50%': ['25.05', 'NaN', '107.41', '1926.13'],
 '75%': ['25.07', 'NaN', '107.61', '1926.20'],
 'max': ['39.00', 'NaN', '190.00', '2004.00'],
 'unique': ['NaN', 1, 'NaN', 'NaN'],
 'top': ['NaN', 0, 'NaN', 'NaN'],
 'freq': ['NaN', 10, 'NaN', 'NaN']}

In [4]:
asmts = src['assessments']
print('Columns in users table:', asmts.keys())
asmts.describe(include=['abdominal_pain', 'brain_fog', 'date','loss_of_smell', 'temperature_f'])

Columns in users table: odict_keys(['abdominal_pain', 'brain_fog', 'date', 'id', 'j_valid_from', 'j_valid_to', 'loss_of_smell', 'temperature_f', 'temperature_f_valid', 'tested_covid_positive', 'user_id'])
fields	 abdominal_pain	      brain_fog	           date	  loss_of_smell	  temperature_f	
count	             30	             30	             30	             30	             30	
unique	              1	              1	            NaN	              1	            NaN	
top	              0	              0	            NaN	              0	            NaN	
freq	             30	             30	            NaN	             30	            NaN	
mean	            NaN	            NaN	  1628912712.34	            NaN	         101.36	
std	            NaN	            NaN	    10077317.46	            NaN	           4.33	
min	            NaN	            NaN	  1613872118.68	            NaN	          95.23	
25%	            NaN	            NaN	  1613975491.70	            NaN	          95.24	
50%	            NaN	

{'fields': ['abdominal_pain',
  'brain_fog',
  'date',
  'loss_of_smell',
  'temperature_f'],
 'count': [30, 30, 30, 30, 30],
 'mean': ['NaN', 'NaN', '1628912712.34', 'NaN', '101.36'],
 'std': ['NaN', 'NaN', '10077317.46', 'NaN', '4.33'],
 'min': ['NaN', 'NaN', '1613872118.68', 'NaN', '95.23'],
 '25%': ['NaN', 'NaN', '1613975491.70', 'NaN', '95.24'],
 '50%': ['NaN', 'NaN', '1614078864.72', 'NaN', '95.26'],
 '75%': ['NaN', 'NaN', '1614182237.74', 'NaN', '95.28'],
 'max': ['NaN', 'NaN', '1644821469.46', 'NaN', '109.64'],
 'unique': [1, 1, 'NaN', 1, 'NaN'],
 'top': [0, 0, 'NaN', 0, 'NaN'],
 'freq': [30, 30, 'NaN', 30, 'NaN']}

## 4.Filtering
Filtering is performed through the use of the apply_filter function. This can be performed on __individual fields__ or at a __dataframe level__. apply_filter applies the filter on data rows.



In [5]:
with Session() as s:
    dst = s.open_dataset('temp2.hdf5', 'w', 'dst')
    df = dst.create_dataframe('df')

    # apply a filter to the dataframe

    filt = (2022 - users['year_of_birth'].data[:]) > 18
    users.apply_filter(filt, ddf=df)  # non-destructive with ddf argument
    print(len(df['id']), ' adults out of ', len(users['id']), ' total subjects found.')

9  adults out of  10  total subjects found.


In [6]:
# Combining filters
# we can make use of fields directly rather than fetching the underlying numpy arrays
# we recommend this approach in general

filt = ((2022 - users['year_of_birth'].data[:]) > 18) & (users['has_diabetes'].data[:] == False)
print(filt)

# fetching numpy arrays
print(users['id'].data[filt])

[ True  True  True  True  True False  True  True  True  True]
[b'0' b'1' b'2' b'3' b'4' b'6' b'7' b'8' b'9']


## 5.Performance boost using numba

As the underlying data is fetched as a numpy array, you can utlize the numba @njit functions to accelarate the data process. For example in the case of summing up symptoms, use a seperate function with @njit decrator can speed up the performance. 

In [9]:
import numpy as np
import time

#sum up the symptoms without njit
test_length = 3000000000  # here we use the a test length rather than 50 rows in the dataset, 
                            # as the difference comes with more rows
symptoms = ['abdominal_pain', 'brain_fog',  'loss_of_smell']
symp_data = {}
for i in symptoms:
    symp_data[i] = np.zeros(test_length, 'int32')
t0 = time.time()
sum_symp = np.zeros(test_length, 'int32')
for i in symptoms:
    sum_symp += symp_data[i]
#print(sum_symp)
print(time.time()-t0)

9.901110410690308


In [10]:
#sum up the symptoms with njit
from numba import njit

@njit
def sum_symptom(symp_data, sum_data):
    sum_data += symp_data
    return sum_data

t0 = time.time()
sum_symp = np.zeros(test_length, 'int32')
for i in symptoms:
    sum_symp = sum_symptom(symp_data[i], sum_symp)
#print(sum_symp)
print(time.time()-t0)  # usually 10x faster dependents on data size

6.684003591537476


## 6.Groupby

The groupby is similar to the groupby api from Pandas dataframe.

In [9]:
with Session() as s:
    dst = s.open_dataset('temp2.hdf5', 'w', 'dst')
    df = dst.create_dataframe('df')
    #drop duplicates
    asmts.drop_duplicates(by = 'user_id', ddf = df)
    print(len(df['user_id']), len(asmts['user_id']))
    
    #count
    df2 = dst.create_dataframe('df2')
    asmts.groupby(by = 'user_id').count(ddf = df2)
    print(len(df2['user_id']), len(asmts['user_id']))
    
    #min/ max
    df3 = dst.create_dataframe('df3')
    asmts.groupby(by = 'user_id').max(target ='date', ddf = df3)
    print(len(df3['user_id']), len(asmts['user_id']))
    df4 = dst.create_dataframe('df4')
    asmts.groupby(by = 'user_id').min(target ='date', ddf = df4)
    print(len(df4['user_id']), len(asmts['user_id']))

    #first/last
    df5 = dst.create_dataframe('df5')
    asmts.groupby(by = 'user_id').first(target ='date', ddf = df5)
    df6 = dst.create_dataframe('df6')
    asmts.groupby(by = 'user_id').last(target ='date', ddf = df6)

10 30
10 30
10 30
10 30


Apart from the groupby, pandas also provide the transform functions. In Transform, the data length is not alterd. Here in ExeTera, we do not have a dedicate API for transform functions, but the same operation can be done via the span:

In [10]:
#transform rather than group by
with Session() as s:
    dst = s.open_dataset('temp2.hdf5', 'w', 'dst')
    df = dst.create_dataframe('df')
    
    symptoms = ['abdominal_pain', 'brain_fog',  'loss_of_smell']
    sum_symp = np.zeros(len(asmts['user_id']), 'int32')
    for i in symptoms:
        sum_symp += np.zeros(len(asmts['user_id']), 'int32')
    
    spans = asmts['user_id'].get_spans()  # make sure asmts dataframe is sorted based on user_id
    max_symp = np.zeros(len(asmts['user_id']), 'int32')
    for i in range(len(spans)-1):
        max_symp[spans[i]:spans[i+1]] = np.max(sum_symp.data[spans[i]:spans[i+1]])
    #write data back to df
    df.create_numeric('max_symp', 'int32').data.write(max_symp)
    print(len(df['max_symp'].data))  # note the field length is the same with transform
    

30


## 7.Join

ExeTera provides functions that provide pandas-like merge functionality on DataFrame instances. We have made this operation as familiar as possible to Pandas users, but there are a couple of differences that we should highlight:


1) merge is provided as a function in the dataframe unit, rather than as a member function on DataFrame instances 


2) merge takes three dataframe arguments, left, right and dest. This is due to the fact that DataFrames are always backed up by a datastore and so rather than create an in-memory destination dataframe, the resulting merged fields must be written to a dataframe of your choosing. 


3) Note, this can either be a separate dataframe or it can be the dataframe that you are merging to (typically left in the case of a "left" merge and right in the case of a "right" merge


4) merge takes a number of optional hint fields that can save time when working with large datasets. These specify whether the keys are unique or ordered and allow the merge to occur without first checking this


5) merge has a number of highly scalable algorithms that can be used when the key data is sorted and / or unique.

In [11]:
from exetera.core.dataframe import merge
with Session() as s:
    dst = s.open_dataset('temp2.hdf5', 'w', 'dst')
    df = dst.create_dataframe('df')
    merge(users, asmts, df, left_on='id', right_on='user_id', how='left')
    print(len(df['id_l'].data))  # note as there are 'id' field in both dataframe, thus a suffix '_l' and '_r'
                                    # are added to the merged dataframe 
    print(df.keys())

30
odict_keys(['FirstName', 'LastName', 'bmi', 'bmi_valid', 'has_diabetes', 'height_cm', 'height_cm_valid', 'id_l', 'j_valid_from_l', 'j_valid_to_l', 'year_of_birth', 'year_of_birth_valid', 'abdominal_pain', 'brain_fog', 'date', 'id_r', 'j_valid_from_r', 'j_valid_to_r', 'loss_of_smell', 'temperature_f', 'temperature_f_valid', 'tested_covid_positive', 'user_id'])


## 8.Sort

In [12]:
from exetera.core.dataframe import merge
with Session() as s:
    dst = s.open_dataset('temp2.hdf5', 'w', 'dst')
    df = dst.create_dataframe('df')
    merge(users, asmts, df, left_on='id', right_on='user_id', how='left')
    s.sort_on(df, df, ('id_l',))

sorted ('id_l',) index in 0.0001354217529296875s
  'FirstName' reordered in 0.2100369930267334s
  'LastName' reordered in 0.0011086463928222656s
  'bmi' reordered in 0.0004885196685791016s
  'bmi_valid' reordered in 0.0004451274871826172s
  'has_diabetes' reordered in 0.0017483234405517578s
  'height_cm' reordered in 0.0004525184631347656s
  'height_cm_valid' reordered in 0.00041365623474121094s
  'id_l' reordered in 0.000408172607421875s
  'j_valid_from_l' reordered in 0.00040650367736816406s
  'j_valid_to_l' reordered in 0.0003733634948730469s
  'year_of_birth' reordered in 0.00042748451232910156s
  'year_of_birth_valid' reordered in 0.0006887912750244141s
  'abdominal_pain' reordered in 0.0015702247619628906s
  'brain_fog' reordered in 0.002073049545288086s
  'date' reordered in 0.0006480216979980469s
  'id_r' reordered in 0.0005962848663330078s
  'j_valid_from_r' reordered in 0.00048804283142089844s
  'j_valid_to_r' reordered in 0.0003993511199951172s
  'loss_of_smell' reordered in

In [13]:
from exetera.core.dataframe import merge
with Session() as s:
    dst = s.open_dataset('temp2.hdf5', 'w', 'dst')
    df = dst.create_dataframe('df')
    merge(users, asmts, df, left_on='id', right_on='user_id', how='left')
    df2 = dst.create_dataframe('df2')
    s.sort_on(df, df2, ('id_l',))

sorted ('id_l',) index in 0.00015783309936523438s
  'FirstName' reordered in 0.003790140151977539s
  'LastName' reordered in 0.003641843795776367s
  'bmi' reordered in 0.002112150192260742s
  'bmi_valid' reordered in 0.0016775131225585938s
  'has_diabetes' reordered in 0.0033807754516601562s
  'height_cm' reordered in 0.0017406940460205078s
  'height_cm_valid' reordered in 0.0017364025115966797s
  'id_l' reordered in 0.0030646324157714844s
  'j_valid_from_l' reordered in 0.0018968582153320312s
  'j_valid_to_l' reordered in 0.001705169677734375s
  'year_of_birth' reordered in 0.0018277168273925781s
  'year_of_birth_valid' reordered in 0.0019350051879882812s
  'abdominal_pain' reordered in 0.003263711929321289s
  'brain_fog' reordered in 0.0048716068267822266s
  'date' reordered in 0.002129793167114258s
  'id_r' reordered in 0.00180816650390625s
  'j_valid_from_r' reordered in 0.0025141239166259766s
  'j_valid_to_r' reordered in 0.001980304718017578s
  'loss_of_smell' reordered in 0.0038

In [14]:
#sorting with an index
with Session() as s:
    dst = s.open_dataset('temp2.hdf5', 'w', 'dst')
    df = dst.create_dataframe('df')
    merge(users, asmts, df, left_on='id', right_on='user_id', how='left')

    index = s.dataset_sort_index((df['id_l'],))

    # apply indices to a destination dataframe
    df2 = dst.create_dataframe('df2')
    df.apply_index(index, df2)
    print(df2['id_l'].data[:])
    
    # apply indices in place
    df.apply_index(index)
    print(df['id_l'].data[:])

[b'0' b'0' b'0' b'1' b'1' b'1' b'2' b'2' b'2' b'3' b'3' b'3' b'4' b'4'
 b'4' b'5' b'5' b'5' b'6' b'6' b'6' b'7' b'7' b'7' b'8' b'8' b'8' b'9'
 b'9' b'9']
[b'0' b'0' b'0' b'1' b'1' b'1' b'2' b'2' b'2' b'3' b'3' b'3' b'4' b'4'
 b'4' b'5' b'5' b'5' b'6' b'6' b'6' b'7' b'7' b'7' b'8' b'8' b'8' b'9'
 b'9' b'9']


## 9. I/O
You can output an ExeTera dataframe back to csv file.

In [15]:
with Session() as s:
    dst = s.open_dataset('temp2.hdf5', 'w', 'dst')
    df = dst.create_dataframe('df')
    merge(users, asmts, df, left_on='id', right_on='user_id', how='left')

    #output a dataframe to to_csv
    df.to_csv('merged.csv')

    #output to csv with row filters
    row_filter = (2022-df['year_of_birth'].data[:]) > 18
    df.to_csv('adults.csv', row_filter)  # save the data you want without change the underlying data in df

    #output to csv with column filters
    df.to_csv('column_filtered.csv', column_filter=['id_l', 'year_of_birth', 'date', 'tested_covid_positive'])  # save the columns you want

In [16]:
!ls *csv

adults.csv  assessments.csv  column_filtered.csv  merged.csv  users.csv


In [17]:
# close src dataset as we open dataset using s=Session()
# this is not necessary if we use context management by with Session as s:
s.close_dataset(src)